In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import matplotlib.dates
import folium
from folium import plugins

In [2]:
from pyIUDX.cat import cat
# Specify the catalogue server details.
# initialize a catalogue class
cat = cat.Catalogue("https://varanasi.iudx.org.in/catalogue/v1")

In [3]:
geo1 = {"circle": {"lat": 25.3176, "lon": 82.9739, "radius": 30000}}
attributes = {"tags": ["dustbin"]}
filters = ["id"]

all_bins_ItemsByID = cat.getManyResourceItems(attributes, filters, geo=geo1)
print(all_bins_ItemsByID[0])
print("Number of items = ", len(all_bins_ItemsByID))

{'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/rs.varanasi.iudx.org.in/varanasi-swm-bins/242'}
Number of items =  219


In [4]:
# Import the item class from pyIUDX.rs
from pyIUDX.rs import item
m = folium.Map(location=[25.3176, 82.9739],zoom_start=12)
plugins.ScrollZoomToggler().add_to(m)
bins = item.Items("https://varanasi.iudx.org.in/catalogue/v1", "https://rs.varanasi.iudx.org.in/resource-server/vscl/v1", all_bins_ItemsByID)
print(bins[0].geoProperties)
for sensor in bins:
  sensor_id = sensor.id  
#   print("Sensor location = ", sensor.location.coordinates)
  folium.Marker([sensor.location.coordinates[1], sensor.location.coordinates[0] ], popup=sensor_id).add_to(m)    
m

ConnectionError: None: Max retries exceeded with url: /catalogue/v1/items/rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/rs.varanasi.iudx.org.in/varanasi-swm-bins/1299 (Caused by None)

In [ ]:
print(bins[0].quantitativeProperties)
print(bins[0].updatedAtIdx)

In [ ]:
bins.during("2020-02-27T00:00:00.000+05:30", "2020-03-05T00:00:00.000+05:30")

In [ ]:
import matplotlib.pyplot as plt

len_arr = []

for sensor in bins:
    len_arr.append([len(sensor.FilledLevel.value[:,1]), sensor])
    
len_arr = np.array(len_arr)
len_arr = len_arr[len_arr[:,0].argsort(kind='mergesort')]
len_arr


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1, figsize=(15,10))
fig.suptitle(bins[0].FilledLevel.name + "\n" + bins[0].FilledLevel.describes, fontsize=20)
plt.xlabel(bins[0].updatedAt, fontsize=18)
plt.ylabel(bins[0].FilledLevel.name, fontsize=16)

idx = -1
sensors = len_arr[idx-1:idx][:,1]
for sensor in sensors:
    plt.plot(sensor.FilledLevel.value[:,sensor.updatedAtIdx], sensor.FilledLevel.value[:,-1], label="Logged Bin_"+sensor.id.split("/")[-1], marker='o')
    plt.vlines(sensor.FilledLevel.value[:,0], 0, 100, label="Cleaned Time")
#     plt.plot(sensor.FilledLevel.value[:,0], sensor.FilledLevel.value[:,-1], label="Cleaned Bin_"+sensor.id.split("/")[-1])
    
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

pts_arr = []
# print("Length ", len(bins))
fig, ax = plt.subplots(1,1, figsize=(15,10))
for sensor in bins:
    pts_arr.append(len(sensor.FilledLevel.value[:,2]))

pts_arr = np.array(pts_arr)
# pts_arr = pts_arr[np.where(pts_arr == 0)]
pts_arr = pts_arr[np.where(np.logical_and(pts_arr > 1, pts_arr < 100))]
print(len(pts_arr))
plt.xlabel("Data packets in one week", fontsize=18)
plt.ylabel("Number of bins", fontsize=16)
ax.hist(pts_arr)
# plt.show()

In [ ]:
pts_arr = []
print("Length ", len(bins))

fig, ax = plt.subplots(1,1, figsize=(15,10))
plt.xlabel("Total garbage collected by bin in one week", fontsize=18)
plt.ylabel("Number of bins", fontsize=16)

for sensor in bins:
    if(sensor.FilledLevel.value.size) >= 0 :
        val = np.sum(sensor.FilledLevel.value[:,2])
        pts_arr.append(val)
    
plt.hist(pts_arr)


In [ ]:
diffarr = np.empty(shape=(1,0))
fig, ax = plt.subplots(1,1, figsize=(15,10))
plt.xlabel("Inter-Arrival Time in hours", fontsize=18)
plt.ylabel("Number of bins", fontsize=16)
for sensor in bins:
    if(sensor.FilledLevel.value.size) > 0 :
        diff = (sensor.FilledLevel.value[1:,1] - sensor.FilledLevel.value[0:-1,1])
        diffarr = np.append(diffarr, diff)

d_hist = [d.total_seconds()/(60*60) for d in diffarr]
ax.hist(d_hist,bins=50)